In [18]:
import os
import pandas as pd
from transformers import BertTokenizer
import six
import re
import json
import numpy as np
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
from transformers import TFElectraModel
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Activation, Lambda, Input
# from tensorflow_addons.layers import CRF
# from crf import CRF
# from keras_contrib.layers import CRF
# from keras_contrib.losses import crf_loss
# from keras_contrib.metrics import crf_viterbi_accuracy

# from tf_crf_layer.layer import CRF
# from tf_crf_layer.loss import crf_loss
# from tf_crf_layer.metrics import crf_accuracy

from tf2crf import CRF, ModelWithCRFLoss
from NER_util import keras_pad_fn

import nltk
from konlpy.tag import Mecab

m = Mecab()

In [2]:
# tokenizer = BertTokenizer.from_pretrained('vocab.txt', do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('../nscc_mecab_base_final/vocab.txt', do_lower_case=False) # 최신 vocab

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [3]:
def df_refine(df):
    # df = df.replace(',', ' ', regex=True)
    df = df.replace('[^ 가-힣0-9a-zA-Z!@#$%^&*.\n,]|[가나다라마바사]\)|[0-9]-?[0-9]?\)', '', regex=True)
    df = df[df['연구분야분류코드1']!='']
    df = df[df['과제_한글키워드']!='']
    df = df[df['연구내용요약']!='']
    df = df[df['연구분야분류코드1']!='-']
    df = df[df['과제_한글키워드']!='-']
    df = df[df['연구내용요약']!='-']
    df = df[df["과제_한글키워드"].str.len() > 1] # 키워드가 한 글자인 경우는 제외 (한 글자로는 옻, 배 2개가 있었음)
    df = df[df["과제_한글키워드"] != '보안'] # 키워드가 보안인 경우는 제외 (요약 내용이 제대로 나오지 않음)
    # df['과제_한글키워드'] = df['과제_한글키워드'].str.replace(' ', '')
    df = df.reset_index(drop=True)
    return df

# word에 한글과 영문 글자가 포함되어 있지 않으면 삭제 해주기
# 입력은 키워드(문자열)
# 반환은 bool, 한글과 영어가 포함된 키워드는 True를 반환, 한글과 영어가 포함되지 않은 키워드라면 False를 반환
def check_hangul(word):
    hangul_pattern = re.compile("["
                                u"\U0000AC00-\U0000D7AF"  # 한글 문자
                                u"\U00000041-\U0000007A"   # 영어 알파벳
                                "]+", flags=re.UNICODE)
    res = hangul_pattern.search(word)
    if res == None:
        return False
    else:
        return True

In [4]:
dir_path = 'jn'

keywords = []
summarys = []
codes = []
for file in os.listdir(dir_path):
    if 'xls' in file:
        df = df_refine(pd.read_excel(os.path.join(dir_path, file)))
        for i in tqdm(range(len(df))):
#         for i in range(500):
            if '99' in df['연구분야분류코드1'].iloc[i]: # 분류되지 않은 태그는 제외
                continue
            keyword = df['과제_한글키워드'].iloc[i].replace(' ,', '').replace(', ', ',')
            summary = re.sub(r"\s{1,}", " ", df['연구내용요약'].iloc[i]) # 2번 이상 반복되는 공백문자를 공백문자 하나로 변경
            if check_hangul(keyword): # 키워드에 한글과 영어가 포함되어 있으면 추가
                keywords.append(keyword)
                codes.append(df['연구분야분류코드1'].iloc[i])
            
            if check_hangul(summary): # 요약에 한글과 영어가 포함되어 있으면 추가
                summarys.append(summary)

100%|██████████| 52372/52372 [00:05<00:00, 10073.65it/s]


In [5]:
print("중복 제거 전 summarys 길이 : ", len(summarys)) # 160625 
summarys = list(set(summarys)) # 요약 중복 제거
print("중복 제거 후 summarys 길이 : ", len(summarys)) # 123725

중복 제거 전 summarys 길이 :  160625
중복 제거 후 summarys 길이 :  123725


In [6]:
df_key_code = pd.DataFrame({"keyword" : keywords, "code" : codes})
df_summary = pd.DataFrame({"summary" : summarys})

idx = []
for i in range(len(df_summary)):
    if '보안상 생략' in df_summary['summary'].iloc[i]:
        idx.append(i)
    elif '세부내용 생략' in df_summary['summary'].iloc[i]:
        idx.append(i)
    elif '세부사항 생략' in df_summary['summary'].iloc[i]:
        idx.append(i)
    elif '보안과제임으로 비공개' in df_summary['summary'].iloc[i]:
        idx.append(i)
    elif '보안과제로 생략' in df_summary['summary'].iloc[i]:
        idx.append(i)
df_summary = df_summary.drop(df_summary.index[idx])
df_summary = df_summary[df_summary['summary'].str.len() > 20] # 요약 길이가 20 이하이면 제외
df_summary = df_summary.drop_duplicates(['summary']) # 요약 중복 제거
df_summary = df_summary.reset_index(drop=True)
print(len(df_summary))

123646


In [7]:
# 요약과 토큰화된 요약 데이터프레임 만들기
token_summarys = []
mecab_summarys = []
for i in tqdm(range(len(df_summary))):
    mecab_summary = m.morphs(df_summary['summary'].iloc[i]) # mecab morphs 적용
    token_summarys.append(tokenizer.tokenize(df_summary['summary'].iloc[i]))
    mecab_summarys.append(tokenizer.tokenize(' '.join(mecab_summary)))
df_summary = pd.DataFrame({"summary" : list(df_summary['summary']), "token_summary" : token_summarys, "mecab_summary" : mecab_summarys})
print("df_summary 길이 : ", len(df_summary))

file_name = "2014-2017_summary_1105.csv"
print("{} 파일 저장".format(file_name))
# df_summary.to_csv(file_name, index=False)
# 요약과 토큰화된 요약 데이터프레임 만들기 끝

100%|██████████| 123646/123646 [12:36<00:00, 163.45it/s]


KeyboardInterrupt: 

In [9]:
each_keywords = []
each_codes = []
for keyword, code in tqdm(zip(keywords, codes), total = len(codes)):
    keyword = keyword.split(',')
    if len(code) < 4: # 코드 길이가 4미만이면 제외
        continue
    for word in keyword:
        # 약간의 정제
        word = word.strip()
        word = word.replace('\n', ' ')
        word = re.sub('[0-9]\.', '', word)
        if len(word) < 2: # 키워드가 2글자 이상이 아니라면 제외
            continue
        if not word: # word가 비어있다면 제외
            continue
        try:
            if len(word.split()[0]) > 10: # 키워드 길이가 10을 초과하면 제외
                continue
        except:
            print("이상한 키워드 : ", (word))
        each_keywords.append(word)
        each_codes.append(code[:4])

100%|██████████| 160624/160624 [00:02<00:00, 71788.11it/s]


In [10]:
# df_keyword = pd.DataFrame({"키워드" : each_keywords, "코드" : each_codes})
df_eachKey_eachCode = pd.DataFrame({"each_keyword" : each_keywords, "each_code" : each_codes})

In [11]:
df_eachKey_eachCode = df_eachKey_eachCode[df_eachKey_eachCode['each_keyword'].str.len() < 20] # 키워드 길이가 20이상이면 제외
df_eachKey_eachCode = df_eachKey_eachCode.reset_index(drop=True) # 인덱스 초기화
print(len(df_eachKey_eachCode)) # 키워드 개수

825457


In [12]:
a = list(set(each_codes))
len(a)

323

In [13]:
# ner_to_index.json 파일 만들기
uniq_codes = list(set(codes))
ner_to_index = ['[CLS]']
ner_to_index.append('[SEP]')
ner_to_index.append('[PAD]')
ner_to_index.append('[MASK]')
ner_to_index.append('O')
for code in uniq_codes:
    ner_to_index.append("B▁{}".format(code))
    ner_to_index.append("I▁{}".format(code))
idx = 0
set_ner_to_index = {}
for ner in ner_to_index:
    set_ner_to_index[ner] = idx
    idx += 1
# with open("ner_to_index_rev.json", "w") as json_file:
#     json.dump(set_ner_to_index, json_file)
# ner_to_index.json 파일 만들기 끝

In [14]:
# with open("ner_to_index_rev.json", 'rb') as f:
#     ner_to_index = json.load(f)

In [15]:
uniq_keywords = []
# for i in tqdm(range(len(df_eachKey_eachCode[df_eachKey_eachCode.duplicated('each_keyword', keep=False)]))):
#     uniq_keywords.append(df_eachKey_eachCode[df_eachKey_eachCode.duplicated('each_keyword', keep=False)].iloc[i]['each_keyword'])
uniq_keywords = list(set(df_eachKey_eachCode['each_keyword'])) # 중복없는 키워드들
print(len(uniq_keywords)) # 240913 -> 232429 -> 232410

232410


In [19]:
dict_keyword_freqdist = {}
for keyword in tqdm(df_eachKey_eachCode['each_keyword']):
    list_keywords = list(df_eachKey_eachCode[df_eachKey_eachCode['each_keyword'] == keyword]['each_code'])
    freqdist = nltk.FreqDist(list_keywords)
    dict_keyword_freqdist[keyword] = freqdist

100%|██████████| 825457/825457 [10:59:41<00:00, 20.85it/s]  


In [20]:
# with open("dict_keyword_freqdist.json", "w") as json_file:
#     json.dump(dict_keyword_freqdist, json_file)

In [21]:
list_keyword = []
list_len_freqdist = []
for keyword in tqdm(df_eachKey_eachCode['each_keyword']):
    list_keyword.append(keyword)
    list_len_freqdist.append(len(dict_keyword_freqdist[keyword]))

100%|██████████| 825457/825457 [00:00<00:00, 932505.25it/s]


In [22]:
df_keyword_code_freq = pd.DataFrame({"keyword" : list_keyword, "freq" : list_len_freqdist})
df_keyword_code_freq = df_keyword_code_freq.drop_duplicates('keyword')
df_keyword_code_freq = df_keyword_code_freq.reset_index(drop=True)
# df_keyword_code_freq.to_csv("keyword-code_freq.csv", index=False)

In [23]:
# # 중복 코드를 갖는 키워드는 빈도가 많은 코드로 태깅
# temp_df = df_eachKey_eachCode[df_eachKey_eachCode.duplicated('each_keyword', keep=False)]
# uniq_codes = []
# for word in tqdm(uniq_keywords):
#     keyword_df = temp_df[temp_df['each_keyword'] == word]
#     if len(keyword_df) < 1: # 중복되는 키워드가 아니라면
# #         print(word)
#         try:
#             key_code = df_eachKey_eachCode[df_eachKey_eachCode['each_keyword'] == word].iloc[0]['each_code']
#         except:
#             print(word)
#             print(df_eachKey_eachCode[df_eachKey_eachCode['each_keyword'] == word])
#         uniq_codes.append(key_code)
#         continue
#     set_code = {}
#     for i in range(len(keyword_df)):
#         code = temp_df[temp_df['each_keyword'] == word].iloc[i]['each_code']
#         if code not in set_code:
#             set_code[code] = 0
#         set_code[code] += 1
# #     key_code = sorted(set_code.items(), key=lambda item:-item[1])[0][0] # 정렬 시간이 꽤 잡아먹을듯... 수정이 필요해보임
#     key_code = max(list(set_code)) # 빈도 수가 같다면 코드의 알파벳이 알파벳 순서상 뒤에 오는 코드로 선택됨
#     uniq_codes.append(key_code)

In [ ]:
# 중복 코드를 갖는 키워드는 빈도가 많은 코드로 태깅
temp_df = df_eachKey_eachCode[df_eachKey_eachCode.duplicated('each_keyword', keep=False)]
list_uniq_keyword = []
list_uniq_code = []
for word in tqdm(uniq_keywords):
    keyword_df = temp_df[temp_df['each_keyword'] == word]
    if len(keyword_df) < 1: # 중복되는 키워드가 아니라면
#         print(word)
        try:
            key_code = df_eachKey_eachCode[df_eachKey_eachCode['each_keyword'] == word].iloc[0]['each_code']
        except:
            print(word)
            print(df_eachKey_eachCode[df_eachKey_eachCode['each_keyword'] == word])
        list_uniq_keyword.append(word)
        list_uniq_code.append(key_code)
        continue
    set_code = {}
    for i in range(len(keyword_df)):
        code = temp_df[temp_df['each_keyword'] == word].iloc[i]['each_code']
        if code not in set_code:
            set_code[code] = 0
        set_code[code] += 1
#     key_code = sorted(set_code.items(), key=lambda item:-item[1])[0][0] # 정렬 시간이 꽤 잡아먹을듯... 수정이 필요해보임
    key_code = max(list(set_code)) # 빈도 수가 같다면 코드의 알파벳이 알파벳 순서상 뒤에 오는 코드로 선택됨

    freq = int(df_keyword_code_freq[df_keyword_code_freq['keyword']==word]['freq'])
    if freq <= 5: # 키워드에 대응되는 분류코드가 5개 이하라면 키워드로 인정
        sort_code_freq = sorted(dict_keyword_freqdist[word].items(), key=lambda item:-item[1])
        if len(sort_code_freq) > 1:
            if sort_code_freq[0][1] >= 100: # 가장 큰 빈도수가 100개 이상이라면 키워드로 인정
                if sort_code_freq[0][1] - sort_code_freq[1][1] > sort_code_freq[0][1]*0.1: # 빈도 수 1순위, 2순위의 차가 1순위의 10%보다 크면 키워드로 인정
                    list_uniq_keyword.append(word)
                    list_uniq_code.append(key_code)
            else:
                if sort_code_freq[0][1] - sort_code_freq[1][1] > 3: # 빈도 수 1순위, 2순위의 차가 3보다 크면 키워드로 인정
                    list_uniq_keyword.append(word)
                    list_uniq_code.append(key_code)
        elif len(sort_code_freq) == 1:
            list_uniq_keyword.append(word)
            list_uniq_code.append(key_code)

  5%|▌         | 11829/232410 [42:10<18:50:28,  3.25it/s] 

In [ ]:
# 키워드, 코드, 토큰화된 키워드 데이터프레임 만들기
list_token_keyword = []
for keyword in tqdm(list_uniq_keyword):
    temp = m.morphs(keyword)
    temp = tokenizer.tokenize(' '.join(temp))
    temp_str = ''
    for i in temp:
        temp_str += i + ' '
    temp_str = temp_str.strip()
    list_token_keyword.append(temp_str)
df_uniq_key_code_token = pd.DataFrame({"keyword" : list_uniq_keyword, "code" : list_uniq_code, "token_keyword" : list_token_keyword})

In [ ]:
df_uniq_key_code_token

In [31]:
# 키워드, 코드, 토큰화된 키워드 데이터프레임 만들기
df_uniq_key_code_token = df_uniq_key_code_token[df_uniq_key_code_token['token_keyword'] != '[UNK]']
df_uniq_key_code_token = df_uniq_key_code_token.drop_duplicates(['keyword']) # 키워드 중복 제거
df_uniq_key_code_token = df_uniq_key_code_token.drop_duplicates(['token_keyword']) # 토큰화된 키워드 중복 제거
df_uniq_key_code_token = df_uniq_key_code_token.sort_values(by=["keyword"],key=lambda x:x.str.len(), ascending=False)
df_uniq_key_code_token = df_uniq_key_code_token.reset_index(drop=True)
print(len(df_uniq_key_code_token))
# df_uniq_key_code_token.to_csv("uniq_keywords, codes, uniq_token_keywords_1107.csv", index=False)

220692


In [32]:
df_uniq_key_code_token

,keyword,code,token_keyword
0,냉매과냉각형 증기분사 다효압축사이클,EF06,냉매 ##과 ##냉 ##각 형 증기 ##분 ##사 다 효 압축 ##사 ##이클
1,포도당양전자 방출 컴퓨터 단층 촬영,OA01,포도 ##당 양 ##전자 방출 컴퓨터 단층 촬영
2,치조골 개조 활성전사체 시퀀싱 분석,LC10,치 ##조 ##골 개 ##조 활성 전사 ##체 시퀀 ##싱 분석
3,최초의 아쿠아리움 홀로그램 구축기술,HE14,최초 의 아 ##쿠 ##아 ##리 ##움 홀로 ##그램 구축 ##기술
4,저강성 플라스틱 표면 구조 성형기술,EC01,저 ##강 ##성 플라스틱 표면 구조 성형 ##기술
...,...,...,...
220687,구름,ND06,구름
220688,치구,EH11,치 구
220689,의미,EE02,의미
220690,지황,LB20,지 ##황


In [33]:
df_uniq_key_code_token.to_csv("uniq_keywords, codes, uniq_token_keywords_1109.csv", index=False)

# --------- 여기까지 정리완료 (RUN ALL ABOVE 가능, 파일이름은 체크 필요) ---------

In [102]:
df_summary

,summary
0,Ground reaction forceG.R.F. 측정 센서의 multi axis ...
1,고령자의 운동 형태별 모션 측정을 통한 균형 능력 향상에 영향을 미치는 동작 인자...
2,1단계 1년차 13년 피부로 호르몬 투여 시에 뇌기능 조절인자 및 피부노화 조절인자...
3,"1단계 1년차 13년 각종 피부세포각질형성세포, 섬유아세포 등에서 합성되는 호르몬 ..."
4,"1년차 근력보조 휠 메커니즘 개발 구동장치 기능 설계, 구동휠 1차 layout, ..."
...,...
123641,1차년도 제2형 당뇨 및 고혈압 등 연관 유전변이 정보 수집 및 분류 GWAS 카...
123642,"생산된 참조에피유전체 데이터의 정도관리 및 참조화 데이터의 coverage, qu..."
123643,만성질환 연관성 마커의 정확도 검증분석targeted fine mapping 혈액...
123644,전당뇨 관련 대사체 마커 발굴 및 추적연구 데이터 생산 13년 학술연구용역으로 생...


In [211]:
df_NER = pd.read_csv("uniq_keywords, codes, uniq_token_keywords.csv")

In [212]:
df_NER = df_NER[df_NER['token_keyword']!='[UNK]'] # 토큰화된 키워드가 '[UNK]'인 행은 제외해줌(150개), 기존 길이 : 235137
df_NER = df_NER.drop_duplicates(['keyword']) # 키워드 중복 제거
df_NER = df_NER.drop_duplicates(['token_keyword']) # 토큰화된 키워드 중복 제거
df_NER = df_NER.reset_index(drop=True)
df_NER.to_csv("uniq_keywords, codes, uniq_token_keywords.csv", index=False)

In [218]:
df_NER[:50]

,keyword,code,token_keyword
0,멤스 마이크로폰 음향센서 마이크 멤스마이크로폰,EA03,멤 ##스 마이크로폰 음향 ##센서 마이크 멤 ##스 ##마이 ##크로 ##폰
1,유연한 전자소자 flexible electronics,EC02,유연 ##한 전자 ##소자 flexible electronics
2,알루미나 다공성 반도체 냉간정수압프레스 고강도,LC03,알루미나 다공성 반도체 냉간 ##정 ##수 ##압 ##프레 ##스 고강도
3,독일어 대명사 조응 관계 해소를 위한 하이브리드 모델,HC07,독일 ##어 대 ##명 ##사 조 ##응 관계 해소 ##를 위한 하이브리드 모델
4,레이더 반사 단면적 감소를 위한 복합재료 구조체 개발,EA15,레이더 반사 단면적 감소 ##를 위한 복합 ##재료 구조 ##체 개발
5,암과 면역질환 치료를 위한 target으로서의 PDK,LA04,암 ##과 면역 ##질환 치료 ##를 위한 target ##으로 ##서 ##의 PDK
6,표면처리 도금액 습식분석 인쇄회로기판 기기분석,EB06,표면 ##처리 도금액 습식 ##분석 인쇄 ##회로 ##기 ##판 기기 ##분석
7,비전통자원 개발을 위한 탄성파탐사의 3D 정량적 해석,EF02,비전 ##통 ##자원 개발 ##을 위한 탄성파 ##탐사 ##의 3 ##D 정량 ##적 해석
8,비트 비트동결건조 비트화장품 비트파우더 항산화,LA09,비트 비트 ##동 ##결 ##건조 비트 ##화 ##장 ##품 비트 ##파 ##우 ##더 항산화
9,개폐기 전압 센싱 배전지능화기기 절연물 접속재,NC05,개폐기 전압 센싱 배전 ##지능 ##화 ##기기 절연물 접속 ##재


In [213]:
temp_summary = pd.read_csv("2014-2017_summary.csv")
temp_summary

,summary
0,금속유기 골격체의 변환을 이용한 금속 및 금속산화물 나노 재료의 새로운 합성법 개발
1,"본 연구개발의 최종목표 달성을 위해 다음의 여덟 가지로 연구 내용을 요약할 수 있음. tyrosine phenollyase, tryptophan indolelyase, OacetylLhomoserine sulfhydrylase, aromatic transaminase를 각각 대장균에 클로닝하고 배양조건을 최적화 하여 타이로신 유도체, 트립토판 유도체, azidohomoalanine, 페닐알라닌 유도체를 인비보로 생합성하는 시스템을 구축함. 비천연아미노산 생합성을 위한 효소와 개량하고자 하는 목적 효소오메가트랜스아미나제, 아민탈수소효소, P4들을 발현할 때, 먼저 비천연아미노산 생산과련 효소를 발현하고, 나중에 목적 산업효소를 발현하는 2단계 단백질 발현 시스템을 구축함. 다중영양요구균주예, Ile, Met, Pro, Trp 그리고 Leu의 영양요구주인 대장균 JE53를 이용하여 여러 종류의 비천연아미노산을 병렬로 효소에 도입하는 다중 잔기특이적 도입방법의 개발, 다중 비천연아미노산이 병렬 도입된 효소에서 활성과 안정성이 개량된 효소의 선별 및 특성 연구, 다중 영양요구균주에 2lth tRNAsynthetase pair를 동시 발현 또는 개량된 synthetase를 이용하여 다중 비천연아미노산의 병렬도입과 동시에 추가적으로 bioorthogonal 반응성이 있는 비천연아미노산의 위치특이적 도입방법의 개발을 통한 고정화가 가능한 효소의 생산 및 개량된 효소의 고정화conjugation 방법 개발, 효소의 mutant library를 구축하고, 비천연아미노산을 도입하여 효소의 특성이 개량된 변이효소를 선별하는 비천연효소공학을 이용한 효소의 방향성 진화, 발효조를 이용한 비천연아미노산이 도입된 효소의 대량생산 방법의 확립, 이렇게 해서 생산된 목적효소트랜스아미나제, 아민탈수소효소, P4를 이용하여 유용성 화합물의 고농도 생산을 위한 반응공정의 확립."
2,"1. 가금티푸스 지속발생 농장 원인균 분리 및 수집 임상 가검물에서의 가금티푸스균 분리 야외 가금티푸스 발생 피해사례 분석 발생농장 임상시료 유래 균 분리 및 수집 닭진드기 및 환경시료에서 가금티푸스균 분리 닭진드기 내외부 오염 균 분리 사료, 먼지 등 환경시료에서 균 분리 야외 병원성균주와 백신균주 감별 동정 분리균에 대한 백신균주 여부 감별 2. 분리균의 유전적 및 생물학적 특성 변화 조사 분리균의 유전자형 변화양상 조사 및 분석 분리균의 PFGE 및 MLVA 유형 분석 분리균의 약제내성 및 생물학적 성상변화 조사 MIC 측정, Vitek 등을 이용한 성상조사 야외분리 생균백신균주의 변이 여부 분석 생균백신주 야외분리주의 유전자형 변화 조사"
3,". CRH 조절물질 screening을 위한 천연물, 유효성분 및 한의학 처방 Library 확보 기존 Library 보강을 위해 MeOH 및 열수 추출물, 한의학처방 추출물 Library 구축 유효성분 Library를 확보 . 선정된 천연물의 모발 성장 촉진 효능 평가 . 선정된 천연물유효성분의 양산화 연구 용매별 효능 검증을 통한 양산화 최적조건 정립 품질 관리를 위한 기준 설정 . 화장품"
4,1. 솔껍질깍지벌레 피해실태 분석 및 방제기술 개발 피해지 발생정밀조사 피해유형 및 발생결과 시기별 발생패턴 분석 솔겁질깍지벌레 페로몬트랩 개발 및 이용 기술 개발 방제기술 개발 저독성 나무주사 방제약제 선발 나무주사 처리시기 구명 2. 솔껍질깍지벌레 방제전략 연구 피해확산 예측 온도별 월동 충의 생존율 조사 온도 조건에 따른 피해확산 잠재력 평가
...,...
122949,1. 1차년도 Lab Scale 통전소성 트리밍 기초 시험 및 물성 평가 Lab Scale 통전소성 트리밍 금형 기술 개발 Lab Scale 센터필라 상단부 전류 분포 해석 기술 개발 전단해석을 통한 전단공정 분석 통전소성 트리밍 금형용 절연체 소재 개발 Lab Scale 통전소성 트리밍 공정 개발 금형 내구성 향상을 위한 스파크 방지 기술 개발 Lab Scale 통전소성 트리밍 개발품 성능 평가 Lab Scale 통전소성 트리밍 기술 개발 2. 2차년도 센터필라 통전소성 트리밍 시편의 물성 평가 통전소성 트리밍 Process map DB 구축 전단공정해석을 통한 시제품 금형 Tool 최적화 1.5GPa급 센터필라 통전소성 트리밍 금형 기술 개발 통전소성 트리밍 공정 개발 1.5GPa급 센터필라 통전소성 트리밍 개발품 성능 평가 1.5GPa급 센터필라 통전소성 트리밍 적용 개발품 정적 강도 평가 1.5GPa급 센터필라 통전소성 트리밍 기술 개발
122950,"골이식재 재료, 지지체 제조공정 자료조사 문헌 및 특허 조사를 통해 유효 약용식물에 대한 개발 가치를 부여하고 인삼 및 약용식물 유래 성분에 대한 신규 기능성 등록을 위한 기초 자료와 가능성을 제시함. 골이식재 재료 및 지지체 제조공정을 문헌 및 특허 조사를 통해 기초 자료를 확립. 골재생 유효성분이 함입된 피브린 스캐폴드 제작 공정 확립 피브린 스캐폴드 제작을 위한 테스트 및 골재생 유효성분이 함입된 피브린 스캐폴드를 제작을 위한 테스트를 통해 피브린 스캐폴드 제작 공정을 확립함. 골재생 유효 성분이 함입된 피브린 스캐폴드의 제형화 시험 인삼 및 유효 약용식물 추출물이 함입된 피브린 스캐폴드 물리적 특성 평가 인삼 및 유효 약용식물 추출물이 함입된 피브린 스캐폴드의 압축강도, 분해시간 등을 평가하여 물리적 특성 평가를 위한 기초 자료를 마련함. 스캐폴드의 열안정성 분석 TGADTGThermogravimetric and Derivative Thermogravimetric 분석은 Pyris 1 TGA analyzer PerkinElmer Inc., USA를 사용하여 수행하였으며, 지속적으로 질소가 흐르는 10 min1의 일정한 가열속도에서 스캔 범위는 50 800로 함. 기계적 특성 스캐폴드의 기계적 특성은 30일 동안 중간엽줄기세포 배양 이전 및 이후에 시험하여 압축하중은 크로스헤드 속도 10 mmmin으로 Intron 3344를 사용하여 각 스캐폴드555 mm에 적용함. 응력변형율 곡선은 각 스캐폴드의 최종 균열 전에 기록하였으며, 압축강도는 시험동안 기록된 최고 압축에서 분석함. 겔, 고형화, 분말 등의 형태로 제형화 함. 인삼 및 유효 약용식물 유래 성분이 함입된 피브린 스캐폴드 시제품 생산 인삼 및 유효 약용식물 유래 성분이 함입된 피브린 스캐폴드 시제품 생산 물리적 특성 평가를 토대로 겔, 고형화, 분말 등의 형태로 생산함. 인삼 복합물질이 함입된 피브린 스캐폴드 시제품 생산 인삼 복합물질이 함입된 피브린 스캐폴드 시제품 생산 물리적 특성 평가를 토대로 겔, 고형화, 분말 등의 형태로 생산함. 3. 연구개발의 추진 가. 연구개발의 추진전략방법 골이식재 재료, 지지체 제조공정 자료조사 골재생 유효성분이 함입된 피브린 스캐폴드 제작 공정 확립 골재생 유효 성분이 함입된 피브린 스캐폴드의 제형화 시험 인삼 및 유효 약용식물 추출물이 함입된 피브린 스캐폴드 물리적 특성 평가 인삼 및 유효 약용식물 추출물이 함입된 피브린 스캐폴드의 압축강도, 분해시간 등을 평가하여 물리적 특성 평가를 위한 기초 자료를 마련함. 겔, 고형화, 분말 등의 형태로 제형화 함. 인삼 및 유효 약용식물 유래 성분이 함입된 피브린 스캐폴드 시제품 생산 인삼 및 유효 약용식물 유래 성분이 함입된 피브린 스캐폴드 시제품 생산 물리적 특성 평가를 토대로 겔, 고형화, 분말 등의 형태로 생산함. 인삼 복합물질이 함입된 피브린 스캐폴드 시제품 생산 인삼 복합물질이 함입된 피브린 스캐폴드 시제품 생산 물리적 특성 평가를 토대로 겔, 고형화, 분말 등의 형태로 생산함."
122951,"바이러스벡터 기반 shRNA 활용 체계 Genomewide arrayed shRNA library 유지관리 다양한 pathway 모듈 라이브러리 추가 제작 렌티바이러스 벡터시스템의 맞춤형 디자인 및 고성능 바이러스벡터 생산 shRNA 라이브러리를 이용한 주요 생물학적 기능별 HCS assay 시스템 구축 및 활용 Pooled shRNA 라이브러리 HTS 시스템 

In [295]:
summary = '1차년도 피부감작성 유발 관련 세포를 이용한 과민성 동물대체시험법 개발 연구 피부감작성 적용을 통한 후보 대체시험법에 대한 예비 평가 실시 표준작업지침서, 예측모델, 용도 등 평가 2차년도 3중단위와 연계한 검증연구 계획 수립 및 추진 피부감작성 동물대체시험법에 대한 검증연구예비평가로 시험법 선정 실험실내 재현성, 실험실간 전수성transferability 입증시험 피부감작성 대체시험법 시험프로토콜 최적화 한국형 인체피부모델 KeraskinVM을 이용한 피부자극시험 및 유세포분석을 이용한 국소림프절시험피부감작성시험 검증연구 정확도 향상을 위한 재시험 및 추가시험 수행 시험법의 OECD 인정을 위한 자료검증연구보고서 등 생산 국내 3개 기관에서 GLP에 준한 검증결과 확보 3차년도 피부감작성 대체시험법 검증연구각질세포주를 이용한 피부감작능 대체시험법 다수의 GLP 실험실이 참여하는 실험실 간 검증연구 수행맹검법 예측력정확성, 민감도, 특이도 평가 자료 생산 3중단위과제와 연계한 검증결과보고서 마련 한국형 인체피부모델 KeraskinVM을 이용한 피부자극시험 검증연구 정확도 향상을 위한 재시험 및 추가시험 수행 OECD 인정을 위한 추적 보완 연구검증연구보고서 등 SkinonaChip의 피부독성 자극성, 감작성 대체시험법 개발 활용 평가를 위한 초기단계 SkinonaChip 개발 연구 SCI 논문게재 4편 이상, 검증연구자료 기록 의무화 및 신뢰성 확보'
summary = '금속유기 골격체의 변환을 이용한 금속 및 금속산화물 나노 재료의 새로운 합성법 개발'
token_summary = tokenizer.tokenize(summary)
keyword = '산화물'
token_keyword = tokenizer.tokenize(keyword)


In [298]:
' '.join(token_summary)
# ''.join(token_keyword)
# token_summary[4]

'금속 ##유기 골격 ##체 ##의 변환 ##을 이용한 금속 및 금속 ##산화 ##물 나노 재료 ##의 새로운 합성법 개발'

In [55]:
df_NER = pd.read_csv("uniq_keywords, codes, uniq_token_keywords.csv")
df_summarys = pd.read_csv('2014-2017_summary.csv')
save_filename = 'NER_ids_label_rev.csv'

keywords = df_NER['keyword']
codes = df_NER['code']
token_keywords = df_NER['token_keyword']
summarys = df_summarys['summary']
# set_keyword_code = dict(zip(keywords, codes)) # 키워드와 코드를 딕셔너리로 만들어줌
set_token_keyword_code = dict(zip(token_keywords, codes)) # 토큰화된 키워드와 코드를 딕셔너리로 만들어줌

In [56]:
# keyword에 한글과 영문 글자가 포함되어 있지 않으면 삭제 해주기
# 입력은 키워드(문자열)
# 반환은 bool, 한글과 영어가 포함된 키워드는 True를 반환, 한글과 영어가 포함되지 않은 키워드라면 False를 반환
def check_hangul(keyword):
    hangul_pattern = re.compile("["
                                u"\U0000AC00-\U0000D7AF"  # 한글 문자
                                u"\U00000041-\U0000007A"   # 영어 알파벳
                                "]+", flags=re.UNICODE)
    res = hangul_pattern.search(keyword)
    if res == None:
        return False
    else:
        return True
idx = []
for i in range(len(df_NER)):
    if not check_hangul(df_NER['keyword'].iloc[i]):
        idx.append(i)

In [57]:
df_NER

,keyword,code,token_keyword
0,멤스 마이크로폰 음향센서 마이크 멤스마이크로폰,EA03,멤 ##스 마이크로폰 음향 ##센서 마이크 멤 ##스 ##마이 ##크로 ##폰
1,유연한 전자소자 flexible electronics,EC02,유연 ##한 전자 ##소자 flexible electronics
2,알루미나 다공성 반도체 냉간정수압프레스 고강도,LC03,알루미나 다공성 반도체 냉간 ##정 ##수 ##압 ##프레 ##스 고강도
3,독일어 대명사 조응 관계 해소를 위한 하이브리드 모델,HC07,독일 ##어 대 ##명 ##사 조 ##응 관계 해소 ##를 위한 하이브리드 모델
4,레이더 반사 단면적 감소를 위한 복합재료 구조체 개발,EA15,레이더 반사 단면적 감소 ##를 위한 복합 ##재료 구조 ##체 개발
...,...,...,...
234877,도시,SD08,도시
234878,랫트,LC07,랫 ##트
234879,벤처,SC07,벤처
234880,애싱,ED02,애 ##싱


In [58]:
df_NER = df_NER.drop(df_NER.index[idx])
df_NER = df_NER.reset_index(drop=True)

In [60]:
df_NER.to_csv("uniq_keywords, codes, uniq_token_keywords.csv", index=False)

In [54]:
len(idx)

24

In [214]:
keywords = df_NER['keyword']
codes = df_NER['code']
token_keywords = df_NER['token_keyword']
summarys = temp_summary['summary']
set_keyword_code = dict(zip(keywords, codes)) # 키워드와 코드를 딕셔너리로 만들어줌

In [34]:
token_summarys = []
for summary in tqdm(summarys[:100]):
    token_summarys.append(tokenizer.tokenize(summary))
token_keywords = []
for keyword in tqdm(keywords):
    temp = tokenizer.tokenize(keyword)
    temp_str = ''
    for i in temp:
        temp_str += i + ' '
    temp_str = temp_str.strip()
    token_keywords.append(temp_str)

100%|██████████| 235185/235185 [00:16<00:00, 14554.44it/s]


In [11]:
set_token_keyword_code = dict(zip(token_keywords, codes)) # 토큰화된 키워드와 코드를 딕셔너리로 만들어줌

In [12]:
set_token_keyword_code

{'멤 ##스 마이크로폰 음향 ##센서 마이크 멤 ##스 ##마이 ##크로 ##폰': 'EA03',
 '유연 ##한 전자 ##소자 flexible electronics': 'EC02',
 '알루미나 다공성 반도체 냉간 ##정 ##수 ##압 ##프레 ##스 고강도': 'LC03',
 '독일 ##어 대 ##명 ##사 조 ##응 관계 해소 ##를 위한 하이브리드 모델': 'HC07',
 '레이더 반사 단면적 감소 ##를 위한 복합 ##재료 구조 ##체 개발': 'EA15',
 '암 ##과 면역 ##질환 치료 ##를 위한 target ##으로 ##서 ##의 PDK': 'LA04',
 '표면 ##처리 도금액 습식 ##분석 인쇄 ##회로 ##기 ##판 기기 ##분석': 'EB06',
 '비전 ##통 ##자원 개발 ##을 위한 탄성파 ##탐사 ##의 3 ##D 정량 ##적 해석': 'EF02',
 '비트 비트 ##동 ##결 ##건조 비트 ##화 ##장 ##품 비트 ##파 ##우 ##더 항산화': 'LA09',
 '개폐기 전압 센싱 배전 ##지능 ##화 ##기기 절연물 접속 ##재': 'NC05',
 '고밀 ##집 센서 ##네 ##트 ##워크 ##에 ##서 ##의 에너지 효율 ##적 충돌 ##완 ##화 기법': 'EE09',
 '일대 ##일 ##로 환경 ##하 ##에 ##서 ##의 동아시아 물류 네트워크 ##의 재 ##구 ##성': 'SC16',
 '가스 ##터 ##빈 제어 복합 ##발전 ##제어 성능 ##시험 성능 ##감 ##시 연소 ##진동': 'EF03',
 '인도네시아 주석 ##광 ##산 ##에 ##서 추출 ##한 히 ##토 ##류 원소 ##로 연구개 ##시': 'EB01',
 '신재 ##생 ##에너지 ##를 만드 ##는 두 ##가지 역할 ##을 하 ##는 모델 시스템': 'EB01',
 '중국 ##의 사물 ##인 ##터 ##넷 ##IoT 표준 ##화 ##를 통한 추격 전략 연구': 'SC16',
 '방부 ##성 다기능 ##계 ##면 ##활성 ##제 고분자 콘크리트 

In [15]:
token_summarys = []
for summary in tqdm(summarys[:5]):
    token_summarys.append(tokenizer.tokenize(summary))

100%|██████████| 5/5 [00:00<00:00, 685.55it/s]


In [60]:
NER_ids = pd.read_csv('NER_ids_label-10000_rev.csv')

In [86]:
# (NER_ids['NER_ids'].iloc[0].find('1'))
(NER_ids['NER_label'].iloc[4])

"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B▁LB03', 'O', 'B▁OC03', 'O', 'O', 'B▁SF01', 'B▁LB03', 'O', 'B▁OA01', 'O', 'O', 'B▁LB03', 'O', 'O', 'B▁OA01', 'O', 'O', 'B▁OA01', 'O', 'B▁OC03', 'O', 'O', 'O', 'O', 'O', 'O', 'B▁NC09', 'O', 'O', 'B▁SF01', 'O', 'B▁LB17', 'O', 'B▁SF01', 'O', 'B▁SF01', 'O', 'O', 'B▁LB03', 'I▁LB03', 'I▁LB03', 'O', 'B▁LB17', 'B▁NB03', 'O', 'B▁EH07', 'O', 'B▁EA10', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B▁LC08', 'O', 'B▁SH01', 'B▁LB03', 'O', 'B▁SG07', 'B▁NC05', 'O', 'O', 'O', 'O', 'O', 'O', 'B▁SC09', 'O', 'O', 'B▁EI03', 'B▁LB03', 'O', 'B▁EF06', 'I▁EF06', 'I▁EF06']"

In [51]:
type(NER_ids['NER_ids'].iloc[0].replace('\n', ''))
# NER_ids['NER_ids'].str.replace('\n', '')
NER_ids['NER_ids'].iloc[i].replace('\n', '')

'[  0   4 289   4   4 183 591   4   4 197   4 637 638 537   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

In [81]:
rev_ids = []

for i in range(len(NER_ids)):
# for i in range((2)):
    chk = False
    temp = NER_ids['NER_ids'].iloc[i].replace('\n', '')[1:-1].split()
    for ids in range(len(temp)):
        if temp[ids] == "1":
            chk = not chk
        if chk:
            if temp[ids] == "0":
                temp[ids] = "2"
    rev_ids.append(temp)

In [90]:
str(rev_ids[0])
df_ids = (rev_ids)
df_summary = NER_ids['summary']
df_label = NER_ids['NER_label']

In [91]:
df = pd.DataFrame({"summary" : df_summary, "NER_ids" : df_ids, "NER_label" : df_label})

In [108]:
df["NER_label"].iloc[0]

"['O', 'B▁LB12', 'O', 'O', 'B▁LB18', 'B▁ND03', 'O', 'O', 'B▁NC10', 'O', 'B▁NC02', 'I▁NC02', 'B▁SF01']"

In [111]:
list_of_ner_ids = keras_pad_fn([df_ids], pad_id=2, maxlen=256)[0]

In [46]:
tokenizer.tokenize(df['summary'].iloc[1])

['본',
 '연구개발의',
 '최종목표',
 '달성을',
 '위해',
 '다음의',
 '여',
 '##덟',
 '가지로',
 '연구',
 '내용을',
 '요약',
 '##할',
 '수',
 '있음',
 '.',
 'tyros',
 '##ine',
 'phen',
 '##oll',
 '##y',
 '##ase',
 '[UNK]',
 'tr',
 '##ypt',
 '##oph',
 '##an',
 'ind',
 '##ole',
 '##ly',
 '##ase',
 '[UNK]',
 'O',
 '##acet',
 '##yl',
 '##L',
 '##hom',
 '##ose',
 '##rine',
 'sulf',
 '##hyd',
 '##ryl',
 '##ase',
 '[UNK]',
 'ar',
 '##omatic',
 'trans',
 '##amin',
 '##ase를',
 '각각',
 '대장균',
 '##에',
 '클로닝',
 '##하고',
 '배양조건',
 '##을',
 '최적화',
 '하여',
 '타이',
 '##로',
 '##신',
 '유도체',
 '[UNK]',
 '트',
 '##립',
 '##토',
 '##판',
 '유도체',
 '[UNK]',
 'a',
 '##z',
 '##id',
 '##oh',
 '##omo',
 '##al',
 '##an',
 '##ine',
 '[UNK]',
 '페',
 '##닐',
 '##알',
 '##라',
 '##닌',
 '유도체를',
 '인',
 '##비',
 '##보',
 '##로',
 '생합성',
 '##하는',
 '시스템을',
 '구축함',
 '.',
 '비',
 '##천연',
 '##아미노산',
 '생합성',
 '##을',
 '위한',
 '효소',
 '##와',
 '개량',
 '##하고자',
 '하는',
 '목적',
 '효소',
 '##오',
 '##메가',
 '##트랜',
 '##스',
 '##아',
 '##미나',
 '##제',
 '[UNK]',
 '아민',
 '##탈',
 '##수소',
 '##효소',
 '[

In [166]:
# df.to_csv("NER_ids_label_rev.csv", index=False)

In [7]:
summarys = list(set(summarys)) # 요약 중복 제거
len(summarys)

123725

In [ ]:
len(codes)

In [12]:
pd.set_option('display.max_colwidth', None) # 전체 텍스트 출력

In [72]:
(df_keyword[df_keyword.duplicated('키워드', keep=False)]['코드']) # 712829

1         LC05
6         LC05
8         LC01
9         LC01
10        LC01
          ... 
832149    LA02
832150    LA02
832151    LA02
832152    LA02
832153    LA02
Name: 코드, Length: 712829, dtype: object

In [100]:
# # 태깅된 키워드와 코드를 파일로 저장
# if len(uniq_keywords) == len(uniq_codes):
#     file_a = open('uniq_keywords, codes.csv', 'w')
#     file_a.write("keyword,code\n")
#     for keyword, code in zip(uniq_keywords, uniq_codes):
#         file_a.write("{},{}\n".format(keyword, code))
# else:
#     file_b = open('uniq_codes.csv', 'w')
#     file_b.write("code\n")
#     for code in uniq_codes:
#         file_b.write("{}\n".format(code))

if len(uniq_keywords) == len(uniq_codes):
    temp_df = pd.DataFrame({"keyword" : uniq_keywords, "code" : uniq_codes})
    temp_df.to_csv("uniq_keywords, codes(rev 전).csv")
else:
    file_b = open('uniq_codes.csv', 'w')
    file_b.write("code\n")
    for code in uniq_codes:
        file_b.write("{}\n".format(code))

In [22]:
# file_a = open('2014-2017_summary(rev 전).csv', 'w')
# file_a.write('summary\n')
# for summary in summarys:
#     file_a.write("{}\n".format(summary))
temp_summary = pd.DataFrame({"summary" : summarys})
temp_summary.to_csv("2014-2017_summary(rev 전).csv", index=False)

In [3]:
temp_summary = pd.read_csv("2014-2017_summary.csv")
temp_summary

,summary
0,금속유기 골격체의 변환을 이용한 금속 및 금속산화물 나노 재료의 새로운 합성법 개발
1,본 연구개발의 최종목표 달성을 위해 다음의 여덟 가지로 연구 내용을 요약할 수 있음...
2,1. 가금티푸스 지속발생 농장 원인균 분리 및 수집 임상 가검물에서의 가금티푸스균 ...
3,". CRH 조절물질 screening을 위한 천연물, 유효성분 및 한의학 처방 Li..."
4,1. 솔껍질깍지벌레 피해실태 분석 및 방제기술 개발 피해지 발생정밀조사 피해유형 및...
...,...
122949,1. 1차년도 Lab Scale 통전소성 트리밍 기초 시험 및 물성 평가 Lab S...
122950,"골이식재 재료, 지지체 제조공정 자료조사 문헌 및 특허 조사를 통해 유효 약용식물..."
122951,바이러스벡터 기반 shRNA 활용 체계 Genomewide arrayed shRNA...
122952,1차년도 피부감작성 유발 관련 세포를 이용한 과민성 동물대체시험법 개발 연구 피부감...


In [111]:
# for i in range(len(temp_summary)):
#     temp_summary['summary']
tokenizer.tokenize(temp_summary['summary'].iloc[0])

['금속',
 '##유기',
 '골격',
 '##체',
 '##의',
 '변환',
 '##을',
 '이용한',
 '금속',
 '및',
 '금속',
 '##산화',
 '##물',
 '나노',
 '재료',
 '##의',
 '새로운',
 '합성법',
 '개발']

In [46]:
df_NER[df_NER['keyword'].str.isalpha()]
# df_NER.sort_values(by=["keyword"],key=lambda x:x.str.len(), ascending=False)

,keyword,code
25754,다기능디스플레이카드,ED04
25764,지불가능주택저렴주택,SB11
25777,노블단백질키나아제씨,LC01
25778,알루미늄나이트라이드,ED01
25782,스텝모터구동집적회로,ED04
...,...,...
235180,도시,SD08
235181,랫트,LC07
235182,벤처,SC07
235183,애싱,ED02


In [39]:
len('수요기반 에너지 관리 시스템 Energy IoT BigData Tree&Platform 에너지 빅데이터 분석 및 모델링 기술 IoT 정보수집 및 전송기술 에너지정보 보안 체계 및 기술')

102

In [41]:
df_NER = df_NER[df_NER['keyword'].str.len() < 30]
df_NER = df_NER.sort_values(by=["keyword"],key=lambda x:x.str.len(), ascending=False)
df_NER = df_NER.reset_index(drop=True)
df_NER.to_csv("uniq_keywords, codes.csv", index=False)

In [65]:
def target_text(keywords, summarys, codes): # 키워드에 코드 태깅 (모든 키워드를 고려하여 텍스트 만듬)
    set_keyword_code = dict(zip(keywords, codes)) # 키워드와 코드를 딕셔너리로 만들어줌
    label_summarys = []
    rev_summarys = []
    res_df = pd.DataFrame()
    for summary in tqdm(summarys):
        label_summary = summary
        rev_summary = summary
        chk = False
        for keyword in keywords:
            if summary.find(keyword):
                chk = not chk
                label_summary = label_summary.replace(keyword, "<{}:{}>".format(keyword, set_keyword_code[keyword][:4]))
                rev_summary = rev_summary.replace(keyword, " {} ".format(keyword)) # summary의 키워드가 다른 단어와 연결되어 토큰화한 label summary와 달라지지 않도록 만들어주기 위한 처리
                rev_summary = re.sub(r"\s{1,}", " ", rev_summary) # 2개 이상의 공백문자를 하나의 공백문자로 수정
        if chk:
            continue
        res_df = pd.DataFrame({'text' : rev_summary, 'label_text' : label_summary})
        res_df.to_csv('NER_text_labelText.csv', index=None, mode='a', encoding='utf-8') # 파일로 하나씩 저장
        label_summarys.append(label_summary)
        rev_summarys.append(rev_summary)
    return rev_summarys, label_summarys

In [9]:
# def target_text(keywords, summarys, codes): # 키워드에 코드 태깅
#     label_summarys = []
#     rev_summarys = []
#     for keyword, summary, code in zip(keywords, summarys, codes):
#         keyword = keyword.split(',')
#         label_summary = summary
#         rev_summary = summary
#         chk = False
#         for word in keyword:
#     #         print(word)
#             if summary.find(word):
#                 chk = not chk
#                 label_summary = label_summary.replace(word, "<{}:{}>".format(word, code[:4]))
#                 rev_summary = rev_summary.replace(word, " {} ".format(word)) # summary의 키워드가 다른 단어와 연결되어 토큰화한 label summary와 달라지지 않도록 만들어주기 위한 처리
#                 rev_summary = re.sub(r"\s{1,}", " ", rev_summary)
# #         label_summary = "[CLS]" + label_summary + "[SEP]"
# #         rev_summary = "[CLS]" + rev_summary + "[SEP]"
#         if chk:
#             continue
#         label_summarys.append(label_summary)
#         rev_summarys.append(rev_summary)
#     return rev_summarys, label_summarys

In [ ]:
# rev_summarys, label_summarys = target_text(keywords, summarys, codes)
rev_summarys, label_summarys = target_text(uniq_keywords, summarys, uniq_codes)
df_summarys = pd.DataFrame(rev_summarys)
df_label_summarys = pd.DataFrame(label_summarys)
# df_keywords = pd.DataFrame(uniq_keywords)
# df_codes = pd.DataFrame(uniq_codes)

  0%|          | 26/160625 [08:53<1356:05:54, 30.40s/it]

In [19]:
df_datas = pd.concat([df_summarys, df_label_summarys], axis=1)
df_datas.columns = ['text', 'label_text']
df_datas = df_datas.drop_duplicates() # 중복 데이터 제거
df_datas = df_datas[df_datas["text"].str.len() > 15] # 과도하게 짧은 데이터 제거
df_datas = df_datas.reset_index(drop=True) # 인덱스 재설정

['유니버셜 스테이지,지면반력,깊이 감지기,센서융',
 '동작 특성인자,운동효과 분석,안정성 평가,사용성 평',
 '피부,호르몬,뇌기능,피부노화,건강기능식',
 '피부,호르몬,대사물질,건강기능식',
 '휠체어,근력보조,핸드림 구동,밸런싱 제어,휠 메커니즘',
 '휠체어,근력보조,핸드림 구동,밸런싱 제어,휠 메커니즘',
 '건강관리플랫폼,유헬스,웰니스,고령자 맞춤형,맞춤형 건강상담',
 '화산폭발재해, 교통재난관리, 항공재난대응, 화산활동 경보, 사회기반시설',
 '화산재, 하천, 환경, 관리, 매뉴얼',
 '화산폭발재해, 재해 보건, 재해 건강, 재해 의료',
 '백두산, 화산폭발',
 '화산, 피해예측, 가시화, 시뮬레이션, 인터페이스',
 '화산재해 대응, 국제 협력 네트워크, 온라인 포럼, 자료은행, 연구지원',
 '백두산, 화산재, 확산모델, 침적량, GIS, 취약도, 직접피해액',
 '장주기지진파, 원거리지진, 화산성지진, 피해저감장치, 피해예측곡선',
 'DMB, 재난방송, 국지적 재난',
 'PGA, 국가지진위험지도, 가속도스펙트럼',
 '성능기반 지진공학, 내진성능평가, 구조물 붕괴 메커니즘, 지진피해예측, 지진취약도',
 '기후변화, 풍수해, 사회적 취약성, 복원탄력성',
 '기후변화, 시나리오, 복합재난, 영향평가, 기후모델링',
 '기후변화, 시설물 방재기준, 복합재해',
 '피해정도 예측, 적절한 초기 대응조치 가능',
 '공공시설, 사유시설, 시설기준, 복구공법, 지속가능 국가복구, 부흥제도',
 '도시 기후환경 변화, 풍해 위험도, 내풍 방재지표, 방재기술',
 'IT, GIS, 풍해위험도 프로그램, 도시풍환경',
 '폭설, 지역별 특성, 도로 제설, 제설취약구간, 제설대책 표준 매뉴얼',
 '폭설 도로 제설, 제설대책 표준 매뉴얼',
 '풍수해, 내수위험도 분석, 풍수해저감 종합계획, 재해저감, 사방',
 '풍수해저감종합계획, 방재계획, 도시계획',
 '분석기법, 방재, 기반구축, 재해영향',
 '지질정보, 지반정보, 통합관리시스템'

In [13]:
df_datas.to_csv('NER_data.csv')

['LC0506',
 'LC05',
 'LC01',
 'LC01',
 'LC0507',
 'LC0507',
 'LC0604',
 'OC03',
 'ND04',
 'OC03',
 'EB02',
 'EE13',
 'ND07',
 'ND07',
 'ND07',
 'EE0701',
 'ND0202',
 'EI0101',
 'ND07',
 'ND07',
 'EI1105',
 'EI1105',
 'EI1105',
 'EH1003',
 'EH10',
 'ND0711',
 'EI1105',
 'EI01',
 'EI01',
 'SA07',
 'ND02',
 'EI03',
 'EI03',
 'ND02',
 'EI03',
 'ND07',
 'ND07',
 'ND0202',
 'ND02',
 'ND02',
 'ND02',
 'EI1105',
 'EI02',
 'EI02',
 'EI0901',
 'EI0903',
 'ND07',
 'ND0701',
 'ED08',
 'ED0803',
 'EA1408',
 'EE13',
 'EA14',
 'EA0501',
 'EA1403',
 'EA1404',
 'EH08',
 'EH0301',
 'EH0301',
 'EH08',
 'EH08',
 'LB0202',
 'LB0202',
 'LB0202',
 'LB0202',
 'LB1301',
 'LB0201',
 'LB0207',
 'LB0202',
 'LB0202',
 'LB0202',
 'LB0204',
 'LB0203',
 'LB0202',
 'LB0202',
 'LB0202',
 'LB0206',
 'LB0206',
 'LB1301',
 'LB1301',
 'LB1301',
 'LB1301',
 'LB0202',
 'LB0202',
 'LB02',
 'LB0202',
 'LB0202',
 'LB0202',
 'LB0202',
 'LB0202',
 'LB0202',
 'LB0202',
 'LB0201',
 'LB0201',
 'LB0202',
 'SC1001',
 'LB02',
 'LB02',


In [68]:
a = pd.read_csv('NER_text_labelText.csv')

In [69]:
a

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        금속 유기 골격체 의 변환 을 이용한 금속 및 금속산화물 나노 재료 의 새로운 합성법 개발    
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                보고된 연구 들과 본 연구 실에서 수행된 선행 연구 에서 lign in 의 축적은 식물 의 면역반응 중 과민성 세포사멸 에 의해서 특이 적으로 유도 된다는 것을 확인 하였다. 따라서, 식물의 병저항성 반 응 과정에서 유도 되는 과민성 세포사멸 과 lign in 의 축적이 어떠한 관계 에 있으며, 과민성 세포사멸 에 있어서의 lign in 의 기능 을 확인 하고자 한다. 먼저, 과민성 세포사멸 현상 에 이상 을 보이는 돌연변이 식물체 를 이용 하여 과민성 세포사멸 의 이상 에 의한 lign in 축적의 변화 를 확인 하고, lign in 축적에 이상 을 보이는 돌연변이 식물체 에서 과민성 세포사멸 현상 의 변화 를 확인 한다. 다음으로 형광물질 이 부착 되어 있는 monolignol을 이용 하여 lign in 의 구조 를 확인 하고, 전자현미경 을 이용 하여 정확한 lign in 의 구조 를 분석 한다. 다음으로 면역반응 에 의한 lign in 의 중합반응 에 관여하는 유전자 를 동정 하고, 이 유전자 에 대한 돌연변이 식물체 에서 병저항성 반응 에 의한 과민성 세포사멸 과 lign in 축적의 변화 를 확인 한다. 이를 통해, 병저항성 과 관련된 lign in 축적의 기작 과 기능 에 대해서 명확하게 규명 하고자 한다.   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [80]:
set_keyword_code['출입문']

nan

In [81]:
len(keywords)

241348

In [13]:
from konlpy.tag import Kkma, Komoran, Okt, Mecab
mecab = Mecab()

In [9]:
summarys[0]

'최근 다양한 이유로 사용되는 GaN소자는 제조공정상의 특성으로 인해서 매우 많은 결함을 포함하고 있다. 이러한 소자는 인공위성과 같은 첨단 장비에도 사용되고 있는데 우주 환경과 같은 극한 환경에서 사용되는 경우에는 결함이 추가적으로 생성되고 기존의 결함과 상호 작용이 발생하게 된다. 소자에 결함이 추가적으로 발생하는 것은 성능의 문제가 발생하거나 오작동의 원인이 된다. 따라서 GaN 제조시에 발생하는 결함의 근본적인 원인을 이론적인 연구를 기반으로 원인을 파악한다. Abinitio와 같은 방법을 활용하여 원자 수준에서 결함에 대한 에너지를 계산하고 결함 구조의 적합성을 확인한다. 내제하는 결함들은 일정한 조건하에서 이동하게 되는데 이러한 과정을 분자동역학 시뮬레이션을 활용하여 이동성을 확인 한다. 여기에 새로이 생성되는 결함과의 관계를 가정한 시뮬레이션을 진행한다. 결함을 이동과 관련한 기초적인 정보를 전위동역학을 통해서 소자에서 발생하는 전위의 이동에 대한 연구를 진행한다. 극한 환경을 가정한 소자 수준의 전산모사를 실시한다. 각 수준에서 연구를 통해서 얻어지는 결과들은 새로운 재료를 개발하거나 소자를 제작하는 과정에서 필요한 설계 변수를 제공한다. 또한 극한 환경에서 사용되는 소자의 신뢰성을 평가하는 기법을 개발한다. 신뢰성 평가 기법을 활용하여 인공위성의 각 부위 구조부, 전자부, 전력부 등에 사용되는 재료에 대한 평가를 실시한다. 이러한 결과를 토대로 로케트나 인공위성과 같은 우주 환경에서 사용되기에 적합한 새로운 재료에 대한 가이드 라인을 제시한다.'

In [72]:
str(mecab.nouns(summarys[0]))
summarys[0]

'최근 다양한 이유로 사용되는 GaN소자는 제조공정상의 특성으로 인해서 매우 많은 결함을 포함하고 있다. 이러한 소자는 인공위성과 같은 첨단 장비에도 사용되고 있는데 우주 환경과 같은 극한 환경에서 사용되는 경우에는 결함이 추가적으로 생성되고 기존의 결함과 상호 작용이 발생하게 된다. 소자에 결함이 추가적으로 발생하는 것은 성능의 문제가 발생하거나 오작동의 원인이 된다. 따라서 GaN 제조시에 발생하는 결함의 근본적인 원인을 이론적인 연구를 기반으로 원인을 파악한다. Abinitio와 같은 방법을 활용하여 원자 수준에서 결함에 대한 에너지를 계산하고 결함 구조의 적합성을 확인한다. 내제하는 결함들은 일정한 조건하에서 이동하게 되는데 이러한 과정을 분자동역학 시뮬레이션을 활용하여 이동성을 확인 한다. 여기에 새로이 생성되는 결함과의 관계를 가정한 시뮬레이션을 진행한다. 결함을 이동과 관련한 기초적인 정보를 전위동역학을 통해서 소자에서 발생하는 전위의 이동에 대한 연구를 진행한다. 극한 환경을 가정한 소자 수준의 전산모사를 실시한다. 각 수준에서 연구를 통해서 얻어지는 결과들은 새로운 재료를 개발하거나 소자를 제작하는 과정에서 필요한 설계 변수를 제공한다. 또한 극한 환경에서 사용되는 소자의 신뢰성을 평가하는 기법을 개발한다. 신뢰성 평가 기법을 활용하여 인공위성의 각 부위 구조부, 전자부, 전력부 등에 사용되는 재료에 대한 평가를 실시한다. 이러한 결과를 토대로 로케트나 인공위성과 같은 우주 환경에서 사용되기에 적합한 새로운 재료에 대한 가이드 라인을 제시한다.'